In [1]:
# pip install geopy
# pip install meteostat

In [2]:
import json
import time
import uuid
import random
from datetime import datetime, timedelta
import pytz
from geopy.geocoders import Nominatim
from meteostat import Point, Hourly

# Geolocalizador

In [3]:
# Definir las coordenadas aproximadas de Medellín
lat_range = (6.217, 6.317)  # Latitud de Medellín
lon_range = (-75.567, -75.467)  # Longitud de Medellín

# Inicializar el geolocalizador
geolocator = Nominatim(user_agent="medellin_locator")

# Obtención de datos

In [4]:
# Función para obtener la comuna y barrio según las coordenadas
def get_location_info(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), language='es')
    address = location.raw['address']
    comuna = address.get('suburb', 'No encontrado')
    barrio = address.get('neighbourhood', 'No encontrado')
    return comuna, barrio

# Generar puntos de venta únicos
def generate_valid_points(n):
    valid_points = []
    while len(valid_points) < n:
        lat = round(random.uniform(*lat_range), 6)
        lon = round(random.uniform(*lon_range), 6)
        comuna, barrio = get_location_info(lat, lon)
        if comuna != 'No encontrado' and barrio != 'No encontrado':
            valid_points.append((lat, lon, comuna, barrio))
    return valid_points

# Generar 100 puntos válidos
points_of_sale = generate_valid_points(100)

# Generar 200 empleados, 2 por cada punto de venta
employees = {i: (f"Employee_{i*2+1}", f"Employee_{i*2+2}") for i in range(100)}

# Función para obtener la temperatura actual
# def get_current_temperature(latitude, longitude):
#     location = Point(latitude, longitude)
#     start = datetime.now() - timedelta(hours=1)
#     end = datetime.now()
#     data = Hourly(location, start, end)
#     data = data.fetch()
#     temperature = data['temp'].mean() if not data.empty else 'No data'
#     return round(temperature, 1)

# Generar datos simulados
def generate_event(pos_index):
    pos = points_of_sale[pos_index]
    worker = employees[pos_index]
    date_now = datetime.now(pytz.timezone('America/Bogota'))
    # temperature = get_current_temperature(pos[0], pos[1])
    
    event = {
        "latitude": pos[0],
        "longitude": pos[1],
        "date": date_now.strftime("%d/%m/%Y %H:%M:%S"),
        "customer_id": random.randint(1000, 2000),
        "employee_id": worker[0] if date_now.weekday() < 5 else worker[1],  # Lunes a Viernes / Fines de Semana
        "quantity_products": random.randint(1, 50),
        "order_id": str(uuid.uuid4()),
        "commune": pos[2],
        "neighborhood": pos[3],
        "partition_date": date_now.strftime("%d%m%Y"),
        "event_date": date_now.strftime("%d/%m/%Y %H:%M:%S"),
        "event_day": date_now.day,
        "event_hour": date_now.hour,
        "event_minute": date_now.minute,
        "event_month": date_now.month,
        "event_second": date_now.second,
        "event_year": date_now.year
    }
    # "current_temperature": temperature
    
    return event

# Enviar los datos
def send_data(s, conn, data):
    conn.sendall(json.dumps(data).encode() + b'\n')
    print("enviado:", data)

# Socket

In [5]:
import socket

In [6]:
# Socket
def main():
    
    host = 'localhost'  # Nombre del host o dirección IP
    port = 7777  # Puerto en el que escucha el servidor

    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    print(f"Listening on {host}:{port}")
    
    conn, addr = s.accept()
    print(f"Connection from {addr}")
    
    try:
        while True:
            event = generate_event(random.randrange(0, 100))
            # Esperar 30 segundos antes de generar el siguiente evento
            send_data(s, conn, event)
            # Guardado en un archivo JSON
            with open('simulated_data.json', 'a') as file:
                file.write(json.dumps(event) + "\n")
            
            time.sleep(5)
            
    except KeyboardInterrupt:
        print("Deteniendo el servidor...")
        conn.close()
        s.close()
        print("Conexión cerrada.")
    
    except BrokenPipeError:
        print("La conexión ha sido cerrada.")
        conn.close()
        s.close()

if __name__ == "__main__":
    main()

Listening on localhost:7777


NameError: name 'conn' is not defined